# KICKSTARTER CAMPAIGN SUCCESS PREDICTION  

* **Date Published**: 2019/08/16
* **Collaborators**: Nateé Johnson & Misha Berrien
* **Data Source**: https://webrobots.io/kickstarter-datasets/

## INTRODUCTION

Kickstarter is a US based global crowd funding platform focused on bringing funding to creative projects. Since the platform’s launch in 2009, the site has hosted over 159,000 successfully funded projects with over 15 million unique backers. Kickstarter uses an “all-or-nothing” funding system. This means that funds are only dispersed for projects that meet the original funding goal set by the creator.

### Project Objective 

Kickstarter earns 5% commission on projects that are successfully funded. Currently, less than 40% of projects on the platform succeed. The objective is to predict which projects are likely to succeed so that these projects can be highlighted on the site either through 'staff picks' or 'featured product' lists.

### Proposed Solutions

1. Predict Successful Campaigns and promote those with the lowest predicted probability of being successful. 
1. Contact creators from those campaigns that are just below the “success” margin and give them insights that will help them succeed. 

## EXPLORATORY DATA ANALYSIS

In [ ]:
# read in kickstarter intermediate data 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns

### Summary Stats

## FEATURE CONSTRUCTION

## MODEL EXPLORATION

## MODEL TUNING

## CONCLUSION & NEXT STEPS